In [1]:
import random
import math
import os
import subprocess
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [2]:

def generate_random_qiskit_circuit(num_qubits, num_operations, filename):
    code = f"""import math
from qiskit import *

q = QuantumRegister({num_qubits}, 'q')
c = ClassicalRegister({num_qubits}, 'c')
qc = QuantumCircuit(q, c)

"""
    
    gates = ["x", "h", "p", "t", "s", "z", "y", "id", "rx", "ry", "rz", "sx", "swap", "rzz", "rxx", "cx", "cz", "ccx", "cswap"]
    
    for _ in range(num_operations):
        gate = random.choice(gates)
        
        if gate in ['x', 'h', 't', 's', 'z', 'y', 'id', 'sx']:
            qubit = random.randint(0, num_qubits - 1)
            code += f"qc.{gate}(q[{qubit}])\n"
        elif gate in ['p', 'rx', 'ry', 'rz']:
            qubit = random.randint(0, num_qubits - 1)
            angle = random.uniform(0, 2*math.pi)
            code += f"qc.{gate}(math.pi/{int(2*math.pi/angle)}, q[{qubit}])\n"
        elif gate in ['cx', 'cz', 'swap']:
            control = random.randint(0, num_qubits - 1)
            target = random.randint(0, num_qubits - 1)
            while target == control:
                target = random.randint(0, num_qubits - 1)
            code += f"qc.{gate}(q[{control}], q[{target}])\n"
        elif gate in ['rzz', 'rxx']:
            qubit1 = random.randint(0, num_qubits - 1)
            qubit2 = random.randint(0, num_qubits - 1)
            while qubit2 == qubit1:
                qubit2 = random.randint(0, num_qubits - 1)
            angle = random.uniform(0, 2*math.pi)
            code += f"qc.{gate}(math.pi/{int(2*math.pi/angle)}, q[{qubit1}], q[{qubit2}])\n"
        elif gate == 'ccx':
            control1 = random.randint(0, num_qubits - 1)
            control2 = random.randint(0, num_qubits - 1)
            target = random.randint(0, num_qubits - 1)
            while control2 == control1 or target == control1 or target == control2:
                control2 = random.randint(0, num_qubits - 1)
                target = random.randint(0, num_qubits - 1)
            code += f"qc.{gate}(q[{control1}], q[{control2}], q[{target}])\n"
        elif gate == 'cswap':
            control = random.randint(0, num_qubits - 1)
            target1 = random.randint(0, num_qubits - 1)
            target2 = random.randint(0, num_qubits - 1)
            while target1 == control or target2 == control or target1 == target2:
                target1 = random.randint(0, num_qubits - 1)
                target2 = random.randint(0, num_qubits - 1)
            code += f"qc.{gate}(q[{control}], q[{target1}], q[{target2}])\n"
    
    with open(filename, 'w') as f:
        f.write(code)

### Generate random circuits in "Testcases" directory

In [5]:
import os
import shutil

# Define base and subdirectories using os.path.join for cross-platform compatibility
base_dir = "Testcases"
deep_dir = os.path.join(base_dir, "deep")
shallow_dir = os.path.join(base_dir, "shallow")

# If they already exist, delete and recreate
for d in (deep_dir, shallow_dir):
    if os.path.isdir(d):
        shutil.rmtree(d)
    os.makedirs(d)

# Generate the random Qiskit circuits in the respective directories
for i in range(3):
    generate_random_qiskit_circuit(3, 3, os.path.join(deep_dir, f"deep_{i}.py"))
    # …and similarly for shallow if needed, e.g.:
    # generate_random_qiskit_circuit(3, 3, os.path.join(shallow_dir, f"shallow_{i}.py"))


In [7]:
import os
import shutil
import subprocess

# Define the config file path.
config = os.path.join("muskit", "generatorConfig.py")

# Create (or reset) the output directory for deep mutants.
mutants_dir = os.path.join("Mutants", "deep")
if os.path.isdir(mutants_dir):
    shutil.rmtree(mutants_dir)
os.makedirs(mutants_dir)

# Define the testcases directory.
testcases_dir = os.path.join("Testcases", "deep")

for file in os.listdir(testcases_dir):
    if file.endswith(".py"):
        name = os.path.splitext(file)[0]
        process = subprocess.Popen(
            [
                "python",
                "-m", "muskit.ComandMain",
                "Create",
                config,
                os.path.join(testcases_dir, file),
                os.path.join(mutants_dir, f"mutants_of_{name}")
            ],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        process.wait()


In [ ]:
import os
import shutil
import subprocess
from tqdm import tqdm

# Build file paths using os.path.join for portability
config = os.path.join("muskit", "executorConfig.py")
inputs = os.path.join("muskit", "testcases.py")
mutants_dir = os.path.join("Mutants", "deep")

# Prepare (or reset) the output directory for bias results
bias_dir = os.path.join("Bias", "deep")
if os.path.isdir(bias_dir):
    shutil.rmtree(bias_dir)
os.makedirs(bias_dir)

for file in tqdm(os.listdir(mutants_dir), desc="Processing all mutants for each shallow circuit"):
    try:
        # Remove the "mutants_of_" prefix to get the name
        name = file[len("mutants_of_"):]
        # Build the save path in the freshly-created bias_dir
        savePath = os.path.join(bias_dir, f"{name}.txt")
        
        # Run the process with the properly constructed paths
        process = subprocess.Popen([
            "python",
            "-m", "muskit.ComandMain",
            "Execute",
            config,
            inputs,
            os.path.join(mutants_dir, file),
            savePath
        ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate()
        if stderr:
            print("Errors:", stderr)
    except Exception as e:
        print(f"Error processing file {file}: {e}")


Processing all mutants for each shallow circuit:  33%|███▎      | 1/3 [01:08<02:17, 68.92s/it]

Mutants/deep/mutants_of_deep_1
Mutants/deep/mutants_of_deep_1
Testcases/deep/deep_1.py
Processing mutant file: Mutants/deep/mutants_of_deep_1/19AddGate_cswap_inGap_1_.py
Original counts: {'100': 471, '000': 529}
Mutant counts: {'000': 511, '100': 489}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_1/15AddGate_rxx_inGap_1_.py
Original counts: {'000': 479, '100': 521}
Mutant counts: {'000': 469, '100': 531}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_1/17AddGate_cz_inGap_1_.py
Original counts: {'100': 503, '000': 497}
Mutant counts: {'000': 477, '100': 523}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_1/38AddGate_cswap_inGap_5_.py
Original counts: {'100': 486, '000': 514}
Mutant counts: {'100': 500, '000': 500}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_1/7AddGate_y_inGap_1_.py
Original counts: {'000': 497, '100': 503}
Mutant counts: {'100': 529, '000': 471}
Calculated bias: 0
Pro

Processing all mutants for each shallow circuit:  67%|██████▋   | 2/3 [02:06<01:02, 62.22s/it]

Mutants/deep/mutants_of_deep_0
Mutants/deep/mutants_of_deep_0
Testcases/deep/deep_0.py
Processing mutant file: Mutants/deep/mutants_of_deep_0/19AddGate_cswap_inGap_1_.py
Original counts: {'000': 1000}
Mutant counts: {'000': 1000}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_0/15AddGate_rxx_inGap_1_.py
Original counts: {'000': 1000}
Mutant counts: {'111': 1000}
Calculated bias: 1.0
Processing mutant file: Mutants/deep/mutants_of_deep_0/17AddGate_cz_inGap_1_.py
Original counts: {'000': 1000}
Mutant counts: {'000': 1000}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_0/38AddGate_cswap_inGap_5_.py
Original counts: {'000': 1000}
Mutant counts: {'000': 1000}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_0/7AddGate_y_inGap_1_.py
Original counts: {'000': 1000}
Mutant counts: {'010': 1000}
Calculated bias: 1.0
Processing mutant file: Mutants/deep/mutants_of_deep_0/24AddGate_s_inGap_5_.py
Original counts: {'000': 1000}

Processing all mutants for each shallow circuit: 100%|██████████| 3/3 [03:19<00:00, 66.40s/it]

Mutants/deep/mutants_of_deep_2
Mutants/deep/mutants_of_deep_2
Testcases/deep/deep_2.py
Processing mutant file: Mutants/deep/mutants_of_deep_2/19AddGate_cswap_inGap_1_.py
Original counts: {'100': 1000}
Mutant counts: {'100': 1000}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_2/15AddGate_rxx_inGap_1_.py
Original counts: {'100': 1000}
Mutant counts: {'100': 510, '010': 490}
Calculated bias: 0.49
Processing mutant file: Mutants/deep/mutants_of_deep_2/17AddGate_cz_inGap_1_.py
Original counts: {'100': 1000}
Mutant counts: {'100': 1000}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_2/38AddGate_cswap_inGap_5_.py
Original counts: {'100': 1000}
Mutant counts: {'100': 1000}
Calculated bias: 0
Processing mutant file: Mutants/deep/mutants_of_deep_2/7AddGate_y_inGap_1_.py
Original counts: {'100': 1000}
Mutant counts: {'000': 1000}
Calculated bias: 1.0
Processing mutant file: Mutants/deep/mutants_of_deep_2/24AddGate_s_inGap_5_.py
Original counts: {

In [10]:
import os
import glob

def analyze_bias_file(path):
    # Read the entire file and strip any trailing comma/newline
    with open(path, 'r') as f:
        data = f.read().strip().rstrip(',')
    # Split on commas and convert to float (skip any empty strings)
    values = [float(x) for x in data.split(',') if x]
    total = sum(values)
    count = len(values)
    avg = total / count if count else 0.0
    return total, count, avg

# Example: process all .txt files in Bias/deep
bias_dir = os.path.join("Bias", "deep")
for filepath in glob.glob(os.path.join(bias_dir, "*.txt")):
    total, count, avg = analyze_bias_file(filepath)
    name = os.path.splitext(os.path.basename(filepath))[0]
    print(f"{name}: sum = {total:.4f}, count = {count}, avg = {avg:.4f}")


deep_0: sum = 11.5090, count = 45, avg = 0.2558
deep_1: sum = 7.9340, count = 55, avg = 0.1443
deep_2: sum = 23.9200, count = 55, avg = 0.4349
